## COG403: Problem 2 of Problem Set 2: Semantic Networks

### All 3 problems for Problem Set 2 Due 1 November 2018, 2 pm

In this question you will be building and experimenting with a semantic network based on the Small World of Words data, to do a partial replication of the Abbott et al. (2015) experiments on a larger free association data set. To build the graph for the semantic network, we will be using the Python library `networkx`. Below are some examples of how to use this library.

In [1]:
import networkx as nx
from provided_functions import plot_graph

# initialize graph
example_graph = nx.DiGraph()

# add nodes
example_graph.add_node('a')
example_graph.add_node('a')
example_graph.add_nodes_from(['b', 'c', 'd'])
example_graph.add_nodes_from(['b', 'c', 'd'])

# add edges
example_graph.add_edge('a', 'b', weight=0.1)
example_graph.add_edge('a', 'c', weight=0.2)
example_graph.add_edge('c', 'd', weight=0.1)
example_graph.add_edge('a', 'd', weight=0.4)

print("example_graph.nodes() => {}".format(example_graph.nodes()))
print("example_graph.edges() => {}".format(example_graph.edges()))
print("'a' in example_graph => {}".format('a' in example_graph))
print("example_graph['a'] => {}".format(example_graph['a']))

# use provided function to plot graph
plot_graph(example_graph)

example_graph.nodes() => ['c', 'd', 'a', 'b']
example_graph.edges() => [('c', 'd'), ('a', 'c'), ('a', 'd'), ('a', 'b')]
'a' in example_graph => True
example_graph['a'] => {'c': {'weight': 0.2}, 'd': {'weight': 0.4}, 'b': {'weight': 0.1}}


/Users/JoshGelua/anaconda3/lib/python3.5/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


<Figure size 640x480 with 1 Axes>

### (a)

Write a function to build a semantic network from the Small World of Words data using `networkx.DiGraph`. Call your function `get_swow_graph` and write it according to the specifications below. Print a list of all the node labels of nodes that *dog* has an outgoing edge to, as well as the weight of each edge.

Note: do NOT print the entire graph, since it is so large.

Note: only words that occur as cues should be included as nodes in the graph.

In [2]:
import networkx as nx
from provided_functions import plot_graph
from data.animals import ANIMAL_TO_CATEGORIES
import csv
from collections import Counter
SWOW_FILE = 'data/SWOW-EN.R100.csv'


def get_swow_graph(node_threshold=5):
    """
    node_threshold: int -- the number of times a word must occur in the SWOW data to be 
    added as a node to your graph.
    
    Generates a directed, weighted networkx.DiGraph where:
        1. Nodes represent cues from the SWOW data that occur at least node_threhsold times.
           Note: the word 'NA' should not occur as a node.
        2. Outgoing edges from each node sum to 1. These should be proportional to the number of times
           each node occurs as a response to the cue associated with the given node.
        3. The node 'animal' is treated a special case. The outgoing edges from the node 'animal' should
           have a uniform probability over all cues in the SWOW data that are keys in the dict
           ANIMAL_TO_CATEGORIES in the data/animals.py file.
    """  
    sum_of_keys = 0
    swow_graph = nx.DiGraph()
    node_weights = {}
    with open(SWOW_FILE, 'rt') as f:
        reader = csv.reader(f)
        SWOW_entries = list(reader)
    for index in SWOW_entries:
        if index[-4] in node_weights:
            node_weights[index[-4]] += 1
        else:
            node_weights[index[-4]] = 1
                                    
    node_weights = {node:weights for node, weights in node_weights.items() if weights >= node_threshold}
    weights = {}
    for node in node_weights:
        weights[node] = []
    with open(SWOW_FILE, 'rt') as f:
        reader = csv.reader(f)
        SWOW_entries = list(reader)
    for index in SWOW_entries:
        if index[-4] in node_weights:
            if index [-3] in node_weights:
                weights[index[-4]].append(index[-3])
            if index [-2] in node_weights:
                weights[index[-4]].append(index[-2])
            if index [-1] in node_weights:
                weights[index[-4]].append(index[-1])
    for node in weights:
        swow_graph.add_node(node)
    for cues in ANIMAL_TO_CATEGORIES:
        if cues in node_weights:
            sum_of_keys += 1
    for cues in ANIMAL_TO_CATEGORIES:
        if cues in node_weights:
            swow_graph.add_edge('animal', cues, weight=(1/sum_of_keys))
    for node in weights:
        connecting_node_weight = []
        length_of_node = len(weights[node])
        if node != 'animal':
            for connecting in weights[node]:
                weight = weights[node].count(connecting) / length_of_node
                if (connecting, weight) not in connecting_node_weight:
                    connecting_node_weight.append((connecting, weight))
            for connecting, weight in connecting_node_weight:
                swow_graph.add_edge(node, connecting, weight=weight)

    return swow_graph
    
    
swow_graph = get_swow_graph()
print(swow_graph['dog'])

{'tired': {'weight': 0.009389671361502348}, 'puppy': {'weight': 0.046948356807511735}, 'bird': {'weight': 0.004694835680751174}, 'happy': {'weight': 0.018779342723004695}, 'walk': {'weight': 0.014084507042253521}, 'nap': {'weight': 0.004694835680751174}, 'caring': {'weight': 0.004694835680751174}, 'animal': {'weight': 0.03755868544600939}, 'bitch': {'weight': 0.004694835680751174}, 'God': {'weight': 0.009389671361502348}, 'friend': {'weight': 0.07042253521126761}, 'tail': {'weight': 0.004694835680751174}, 'hair': {'weight': 0.004694835680751174}, 'days': {'weight': 0.009389671361502348}, 'stubborn': {'weight': 0.004694835680751174}, 'log': {'weight': 0.004694835680751174}, 'faithful': {'weight': 0.004694835680751174}, 'mate': {'weight': 0.004694835680751174}, 'ball': {'weight': 0.009389671361502348}, 'frog': {'weight': 0.004694835680751174}, 'fish': {'weight': 0.004694835680751174}, 'fear': {'weight': 0.004694835680751174}, 'fun': {'weight': 0.009389671361502348}, 'elephant': {'weight'

### (b)

In this step, you will visualize your graph using the function `plot_graph` in `provided_functions.py`. Because the actual graph has thousands of nodes, we will visualize a portion of the graph. Write a function `get_subgraph` based on the specifications below. Call `get_subgraph` on your graph with three start nodes: *dog*, *turtle*, and *animal*. For *dog* and *turtle*, use `length=2` and `threshold=0.05`. For *animal*, use `length=1` and `threshold=0`. Call `plot_graph` on each subgraph.

Hint: use the `subgraph` method of networkx graphs.

Note: By default, the `plot_graph` function plots a graph as undirected. For your graphs for this question, please use the default to print them without the arrows. If you are curious and want to experiment, you can make it show the arrows by setting `arrows=True`. The reason it's set to `False` by default is that the way `matplotlib` renders the arrows decreases the interpretability of the graph.

In [3]:
from provided_functions import plot_graph
 
 
def get_subgraph(G, start, length=2, threshold=0.05):
    """
    G: networkx.DiGraph -- semantic network of SWOW data
    start: str -- name of node to start with
    length: int -- the maximum distance a node can be from start to be included in the result.
        The length of a path through the graph is the number of edges needed to get from
        the start node to the end node (e.g. the path from A->B->C is 2).
    threshold: float -- the minimum edge weight required for an edge to be added to the graph
    
    Return a subgraph of G based on a search starting at start. Only include nodes in your graph that 
    have a distance of length or less from your start node. When searching, only add nodes that are
    connected to your graph by an edge with weight threshold or higher.
    """
    sg = G[start]
    nodes = [start]
    
    for k in sg:
        if sg[k]['weight'] >= threshold:
            nodes.append(k)
            
    i = 1
    while i < length:
        for node in nodes[1:]:
            subg = G[node]
            for k in subg:
                if subg[k]['weight'] >= threshold and k not in nodes:
                    nodes.append(k)
        i += 1
    
    
    return G.subgraph(nodes)

turtle_graph = get_subgraph(G, 'turtle', length=2, threshold=0.05)
dog_graph = get_subgraph(G, 'dog',length=2, threshold=0.05)
animal_graph = get_subgraph(G, 'animal', length=1, threshold=0)

plot_graph(turtle_graph)
plot_graph(dog_graph)
plot_graph(animal_graph)

NameError: name 'G' is not defined

### (c)

Write two functions `get_most_likely_walk` and `get_least_likely_walk` that find the most and least likely walks from a given start node. Implement your functions according to the docstrings below.

Call your functions with the graph from part a. In both cases, set `start = 'dog'` and `walk_length = 10`.

In [ ]:
def get_most_likely_walk(G, start, walk_length):
    """
    G: networkx.DiGraph -- semantic network of SWOW data
    start: str -- name of node to start with
    walk_length: the length of the walk to return.
    
    Return a list of length walk_length representing a walk. Do not include start in your
    result. Each node in your walk should be most likely node given the previous node. In
    other words, it should be the node that the previous node has the highest weighted
    outgoing edge to.
    
    Do not allow repeats in your walk. For example, say 'fox' is your start node and
    'den' is the most likely node to follow 'fox'. Even if 'fox' is also the most
    likely node to follow 'den', it should not be revisited after we've seen it. In this case,
    you should select the second highest weighted edge.
    """
    walk = []
    point = G[start]
    prob = 0
    for steps in point:
        if point[steps]['weight'] > prob:
            prob = point[steps]['weight']
            step = steps
    walk.append(step)
    for i in range(walk_length-1):
        after_step = G[step]
        prob = 0
        for steps in after_step:
            if after_step[steps]['weight'] > prob and steps not in walk and steps != start:
                mw = after_step[steps]['weight']
                step = steps
        walk.append(step)
    
    return walk 
print("Starting at dog, the most likely walk is:")
print(get_most_likely_walk(G, 'dog', 10))

def get_least_likely_walk(G, start, walk_length):
    """
    G: networkx.DiGraph -- semantic network of SWOW data
    start: str -- name of node to start with
    walk_length: the length of the walk to return.
    
    Return a list of length walk_length representing a walk. Do not include start in your
    result. Each node in your walk should be the least likely node that the previous node has an
    outgoing edge to. (Note that there will be nodes that have a zero probability given
    the previous node, and are not connected to the previous node at all. These should
    not be included.)
    
    Do not allow repeats in your walk. For example, say 'fox' is your start node and
    'rock' is the least likely node to follow 'fox'. Even if 'fox' is also the least
    likely node to follow 'rock', it should not be revisited after we've seen it. In
    this case, you should select the second lowest weighted edge.
    """
    walk = []
    point = G[start]
    prob = 1
    for steps in point:
        if point[steps]['weight'] < prob:
            prob = point[steps]['weight']
            step = steps
    walk.append(step)
    for i in range(walk_length-1):
        after_step = G[step]
        prob = 1
        for steps in after_step:
            if after_step[steps]['weight'] < prob and steps not in walk and steps != start:
                mw = after_step[steps]['weight']
                step = steps
        walk.append(step)
    
    return walk 
print ("Starting at dog, the least likely walk is:")
print(get_least_likely_walk(G, 'dog', 10))

### (d)

Write a function `weighted_random_walk` to generate a random walk of length `walk_length` starting at node `start`. The probability of choosing the next node at each step in the walk should be based on your edge weights. You can implement this using `np.random.choice` (take a look at the documentation for the parameter `p`, which allows you to pass a list of weighted probabilities for the items to select from). These random walks (unlike in part c above) should allow revisiting nodes that have been visited before (since this is allowed in the Abbott et al. work).

Your function should return a list of strings representing nodes visited. Implement your function according to the docstring below.

Print the list of strings for a sample walk using the graph from part a with `start = 'animal'` and `walk_length = 50`.

In [ ]:
def weighted_random_walk(G, start, walk_length):
    """
    G: networkx.DiGraph -- semantic network of SWOW data
    start: str -- name of node to start with
    walk_length: the length of the walk to return.
    
    Return a list of length walk_length representing a walk. Do not include start in your
    result. Each node in your walk should be randomly selected based on the weights of the
    outgoing edges of the previous node. A node can be visited more than once.
    """
    # TODO
    pass

### (e)

Write a function `get_animals_and_IRT` according to the docstring below. Your function should return a tuple of (description, animal_list), where:
 * description is a string description. Each line should include a valid `item`, as well as the `steps` between the item and the previous item (ie, the nodes on the path between each pair of responses). A response is a valid item if its label is in `ANIMAL_TO_CATEGORIES` (defined in `data/animals.py`) and it has not been seen previously in the walk.
 * animal_list is a list of tuples of `(item, IRT)`, where item is a valid item, and the IRT is the interitem response time, as defined in Abbott et al.
 
For example, the walk `[animal, dog, pet, cat, dog, bone, dinosaur, lizard]` 

The description would be:

`item: dog       steps:
 item: cat       steps: pet
 item: lizard    steps: dog, bone, dinosaur`

And the animal_list would be:
`[('dog', 1), ('cat', 2), ('lizard', 4)]`

Note that `dog` occurs twice in the random walk, but is only listed as an `item` one time.  Also, we know `dinosaur` is an animal, but it's not listed in `ANIMAL_TO_CATEGORIES`.

Run your function `weighted_random_walk` from part d (with `start = 'animal'` and `walk_length = 50`) until you find a walk with at least 6 valid `items`. Print the random walk, as well as the description and the animal_list returned by `get_animal_and_IRT` called on this walk.

In [ ]:
from data.animals import ANIMAL_TO_CATEGORIES


def get_animal_and_IRT(node_list):
    """
    node_list: list of string
    
    Return a tuple (description, animal_list). Where:
      - description is a string in the format shown in the question description above
      - animal_list is a list of tuples of (item, IRT), where item is a unique item, and count
          is the interitem response time, as defined in Abott et al.
    Ignore the nodes visited after the last unique item is found.
    """
    # TODO
    pass

### (f)

Here, you'll do an analysis of the IRT pattern in the random walks you generate, somewhat simplified from the Abbott et al. reading.  

First, you'll need to recognize "patch switches": Check for each valid animal item whether it shares a category with the previous valid animal item in the walk.

Second, you'll calculate the mean IRT across the entire walk (mIRT), as well as the mean difference from mIRT at each of three points: At valid animals that constitute a patch switch (position 1), as well as at valid animals one before a patch switch (position -1), and valid animals one after a patch switch (position 2).

Finally, you'll average these values across a set of random walks.

Implement this in the `graph_patch_switches` function below, according to the docstring.  (Note that the walks you pass into the function will be in the format of a list of (item, IRT) tuples returned by the function `get_animal_and_IRT` in part e).

Call `graph_patch_switches` on the two input lists provide in the file `sample_rw_lists.py` and show the graph for each.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from data.animals import ANIMAL_TO_CATEGORIES
from sample_rw_lists import sample1, sample2


def generate_plot(triple_averages):
    """
    triple_averages: list of length three representing the difference in mean IRTs at "patch switch"
        positions compared to the overall mean IRT.
        The order should be (position -1, position 1, position 2).
        
    Generate a bar graph of triple_averages.
    """
    y_positions = np.arange(len(triple_averages))
    plt.bar(y_positions, triple_averages, align='center', alpha=0.5)
    plt.xticks(y_positions, ['position -1', 'position 1', 'position 2'])
    plt.ylabel('difference from mean IRT')
    plt.show()


def graph_patch_switches(walks):
    """
    walks: list of list of tuple. Each inner list represents a walk. Each tuple in a walk
        should be of the format (item, IRT), where item is a string representing a valid animal
        in a walk and IRT is an integer indicating the time between this item and the previous one.
    
    Generate a graph of the average difference between overall mean IRT of a walk, and the mean IRT
    at each of three patch switch positions:
        position -1 corresponds to the items just before the first item in a patch switch, 
        position 1 corresponds to the first items in a patch switch, 
        position 2 corresponds to the items in the next position following those in position 1,
            when those next items are in the same patch as the item in position 1.
    Compute the mean IRT for each of the three positions, and the difference between those means
        and the overall mean IRT of each walk.
    Calculate the mean of these differences across all the random walks and plot those using the
        generate_plot function defined above.
    
    """
    # TODO
    pass

### (g)

Now call `graph_patch_switches` on a list of N walks generated using the graph from part a with `start = 'animal'` and `walk_length = 50`. Only include walks containing 6 or more valid animal items in your analysis. (More specifically, you should generate walks until you have N walks that contain at least 6 valid animal items.)

(i) Run your function with N=20 and show the graph.  Explain whether this matches the patch switch pattern seen in the human data and replicated by Abbott et al.

(ii) Now run your function 10 more times with N=20.  Discuss whether the graphs across these different runs show the same pattern consistently.  Please do not include these graphs in what you turn in.

(iii) Run your function two times with N=1000 random walks (this can take about 2 minutes each), and print the graphs for each of these two runs.  Explain whether the graphs match the patch switch pattern seen in the human data and replicated by Abbott et al.  Are the results consistent across the two runs?

### (h)

Write a script to call `get_animals_and_IRT` from part e on 20 random walks of length 50 starting at 'animal'. Write your script so that each random walk contains at least 6 valid animal items. Make sure to clearly number walks 1 - 20.

### (i)

(i) Name three different aspects of the paths you see in part h which contribute to the length of a path and which are not simple, direct associations with animal words.  For example, paths such as cat=>pet=>goldfish or tiger=>stripes=>zebra seem to match our intuition about what might trigger chains of responses in human semantic fluency. But you almost certainly found some paths in which chains of associations are longer due to some interesting factors.  Please identify three of these and show an example of each from your paths.

(ii) Also, find a path that you think is particularly funny, and say why.

(iii) How does your inspection of the 20 paths you generated influence your assessment of the findings in Abbott et al.? 